### setup

In [1]:
import gzip
import json
from typing import List
from collections import Counter, defaultdict
from tqdm import tqdm
import os
import inspect_ai
from openai import OpenAI
from transformers import AutoTokenizer, AutoModelForCausalLM
from huggingface_hub import login
from dotenv import load_dotenv
import torch as t
import subprocess
import contextlib
import shutil
import ast
import textwrap

In [2]:
from inspect_ai import Task, task
from inspect_ai.dataset import Sample, hf_dataset
from inspect_ai.util import ExecResult, sandbox
from inspect_ai.scorer import CORRECT, INCORRECT, Score, Scorer, Target, accuracy, scorer, stderr
from inspect_ai.solver import TaskState, generate
from inspect_ai.model import get_model
from inspect_ai.log import read_eval_log
import re

In [3]:
load_dotenv()
login(token = os.environ['HF_TOKEN'])

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [4]:
IMPORT_HELPER = {
    "python": [
        "import math",
        "import re",
        "import sys",
        "import copy",
        "import datetime",
        "import itertools",
        "import collections",
        "import heapq",
        "import statistics",
        "import functools",
        "import hashlib",
        "import numpy",
        "import numpy as np",
        "import string",
        "from typing import *",
        "from collections import *",
    ],
    "go"    : [
        "math",
        "strings",
        "fmt",
        "strconv",
        "time",
        "bytes",
        "regexp",
        "sort",
        "math/rand",
        "crypto/md5",
    ],
    "cpp"   : [
        "#include<stdlib.h>",
        "#include<algorithm>",
        "#include<math.h>",
        "#include<stdio.h>",
        "#include<vector>",
        "#include<string>",
        "#include<climits>",
        "#include<cstring>",
        "#include<iostream>",
    ],
}

# instruction prepended to code problem
INSTRUCTION = """
Read the following function signature and docstring, and fully implement
the function described. Your response should only contain the code for
this function.\n
"""

LANG_PREFIX = {
    "cpp"          : "// language: C++",
    "java"         : "// language: Java",
    "js"           : "// language: JavaScript",
    "javascript"   : "// language: JavaScript",
    "go"           : "// language: Go",
    "python"       : "# language: Python",
}

In [5]:
model = get_model(
        'hf/meta-llama/Llama-3.1-8B-Instruct', 
        device = 'auto',
        torch_dtype=t.bfloat16,
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

### Data Processing

In [6]:
def stream_jsonl_all(filename: str):
    results = []
    fp = gzip.open(open(filename, "rb"), "rt")
    for line in fp:
        if any(not x.isspace() for x in line):
            results.append(json.loads(line))
    fp.close()

    return results

In [7]:
python_content = stream_jsonl_all('data/python_data.gz')
cpp_content = stream_jsonl_all('data/cpp_data.gz')
go_content = stream_jsonl_all('data/go_data.gz')
java_content = stream_jsonl_all('data/java_data.gz')
js_content = stream_jsonl_all('data/js_data.gz')
content = [python_content, cpp_content, go_content, java_content, js_content]

In [8]:
generations = stream_jsonl_all('data/python_generations.gz')
generations[0]['generation']

'    for idx, elem in enumerate(numbers):\n        for idx2, elem2 in enumerate(numbers):\n            if idx != idx2:\n                distance = abs(elem - elem2)\n                if distance < threshold:\n                    return True\n\n    return False\n'

In [9]:
for lang in content:
    print(lang[0].keys())
    print()

dict_keys(['task_id', 'prompt', 'canonical_solution', 'test', 'text', 'declaration', 'example_test'])

dict_keys(['task_id', 'prompt', 'canonical_solution', 'test', 'declaration', 'example_test'])

dict_keys(['task_id', 'prompt', 'import', 'docstring', 'declaration', 'canonical_solution', 'test', 'test_setup', 'example_test'])

dict_keys(['task_id', 'prompt', 'canonical_solution', 'test', 'text', 'declaration', 'example_test'])

dict_keys(['task_id', 'prompt', 'canonical_solution', 'test', 'declaration', 'example_test'])



### LLM output => code  
*I need to see LLM output & transform that into code*

#### Find Code

In [ ]:
def find_code_new(completion: str) -> str:
    pattern_1 = re.compile(r"```java\n(.*?)```", re.DOTALL)
    pattern_2 = re.compile(r"```\n(.*?)```", re.DOTALL)
    matches = pattern_1.findall(completion) + pattern_2.findall(completion)
    if len(matches) == 0: raise Exception('error processing model output')

    extracted_answer = matches[0]
    extracted_answer = naive_extract_function(extracted_answer)
    return str(extracted_answer)

def naive_extract_function(code: str) -> str:
    lines = code.splitlines()
    found = False
    for line in lines:
        if line.lstrip().startswith('const'):
            lines.remove(line)
            found = True
            break  

    if not found: 
        print('error extracting function body')
        return 'error msg'
    return "\n".join(lines)
    

#### Run Eval

In [ ]:
@scorer(metrics=[accuracy(), stderr()])
def verify() -> Scorer:
    async def score(state: TaskState, target: Target) -> Score:
        idx = state.sample_id
        model_completion = state.output.completion
        processed_completion = ""

        val = CORRECT
        # if 'error' in processed_completion:
        #     print(f'error in sample: {idx}')
        #     val = INCORRECT

        # final_code = [
        #     state.metadata["prompt"],
        #     processed_completion,
        #     "\n\n",
        #     state.metadata["test"],
        # ]

        return Score(
            value=val,
            metadata={
                'completion': model_completion,
                'processed': processed_completion,
                # 'final_code': ''.join(final_code),
                'idx': idx
            },
        )

    return score

In [150]:
lang = 'java'

def humaneval_record_to_sample(record):
    model_input = INSTRUCTION + LANG_PREFIX[lang] + '\n' + record['prompt'] 
    
    return Sample(
        id=record["task_id"],
        input=model_input,
        target=record["canonical_solution"],
        metadata={
            "prompt": record["prompt"],
            "test": record["test"],
        },
    )

humaneval_dataset = hf_dataset(
    path = 'THUDM/humaneval-x',
    name = lang,
    split = 'test',
    sample_fields = humaneval_record_to_sample,
    trust = True,
)

Loading dataset THUDM/humaneval-x from Hugging Face...


0000.parquet:   0%|          | 0.00/117k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/164 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/164 [00:00<?, ? examples/s]

In [ ]:
samples = 10

@task
def humaneval():
    return Task(
        dataset = humaneval_dataset[:samples],
        solver = generate(),
        scorer = verify(),
        sandbox = 'local',
    )

In [153]:
epochs = 1
inspect_ai.eval(humaneval(), model = model, epochs = epochs)

Output()

#### Code Extraction Tests

In [154]:
idx = 0

js_eval = read_eval_log('logs/java_1.eval')
data = js_eval.samples[idx].scores['verify'].metadata
print(data['completion'])
# print(data['processed'])
# print(data['final_code'])

# find_code_new(data['completion'])

```java
import java.util.*;

public class Solution {
    /**
     * Check if in given list of numbers, are any two numbers closer to each other than given threshold.
     *
     * @param numbers A list of numbers to check
     * @param threshold The minimum distance between two numbers to be considered as close
     * @return True if any two numbers are closer than the threshold, False otherwise
     */
    public boolean hasCloseElements(List<Double> numbers, double threshold) {
        if (numbers.size() < 2) {
            return false;
        }

        for (int i = 0; i < numbers.size() - 1; i++) {
            for (int j = i + 1; j < numbers.size(); j++) {
                if (Math.abs(numbers.get(i) - numbers.get(j)) < threshold) {
                    return true;
                }
            }
        }

        return false;
    }
}
```


In [158]:
print(js_content[0]['test'])

const testHasCloseElements = () => {
  console.assert(hasCloseElements([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.3) === true)
  console.assert(
    hasCloseElements([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.05) === false
  )
  console.assert(hasCloseElements([1.0, 2.0, 5.9, 4.0, 5.0], 0.95) === true)
  console.assert(hasCloseElements([1.0, 2.0, 5.9, 4.0, 5.0], 0.8) === false)
  console.assert(hasCloseElements([1.0, 2.0, 3.0, 4.0, 5.0, 2.0], 0.1) === true)
  console.assert(hasCloseElements([1.1, 2.2, 3.1, 4.1, 5.1], 1.0) === true)
  console.assert(hasCloseElements([1.1, 2.2, 3.1, 4.1, 5.1], 0.5) === false)
}

testHasCloseElements()



In [106]:
print(js_eval.samples[0].input)


Read the following function signature and docstring, and fully implement
the function described. Your response should only contain the code for
this function.

// language: JavaScript
/* Check if in given list of numbers, are any two numbers closer to each other than
  given threshold.
  >>> hasCloseElements([1.0, 2.0, 3.0], 0.5)
  false
  >>> hasCloseElements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)
  true
  */
const hasCloseElements = (numbers, threshold) => {



In [ ]:
# preprocess – get 5 evals files!
num_eval_logs = 5
eval_logs = []
for i in range(num_eval_logs):
    file_name = f'logs/test_{i}.eval'
    eval_logs.append(read_eval_log(file_name))

samples_read = 164
for i in range(samples_read):
    processed_results = []
    for j in range(num_eval_logs):
        data = eval_logs[0].samples[i].scores['verify'].metadata
        processed_results.append(data['processed'])
    
    assert len(set(processed_results)) == 1

# if this works, easy to scale it up to 164 samples.

#### Working Extractors

In [ ]:
# JAVASCRIPT

def find_code_new(completion: str) -> str:
    pattern_1 = re.compile(r"```javascript\n(.*?)```", re.DOTALL)
    pattern_2 = re.compile(r"```\n(.*?)```", re.DOTALL)
    matches = pattern_1.findall(completion) + pattern_2.findall(completion)
    if len(matches) == 0: raise Exception('error processing model output')

    extracted_answer = matches[0]
    extracted_answer = naive_extract_function(extracted_answer)
    return str(extracted_answer)

def naive_extract_function(code: str) -> str:
    lines = code.splitlines()
    found = False
    for line in lines:
        if line.lstrip().startswith('const'):
            lines.remove(line)
            found = True
            break  

    if not found: 
        print('error extracting function body')
        return 'error msg'
    return "\n".join(lines)
    

@scorer(metrics=[accuracy(), stderr()])
def verify() -> Scorer:
    async def score(state: TaskState, target: Target) -> Score:
        idx = state.sample_id
        model_completion = state.output.completion
        processed_completion = find_code_new(model_completion)

        val = CORRECT
        if 'error' in processed_completion:
            print(f'error in sample: {idx}')
            val = INCORRECT

        final_code = [
            state.metadata["prompt"],
            processed_completion,
            "\n\n",
            state.metadata["test"],
        ]

        return Score(
            value=val,
            metadata={
                'completion': model_completion,
                'processed': processed_completion,
                'final_code': ''.join(final_code),
                'idx': idx
            },
        )

    return score

In [ ]:
# PYTHON

def find_code_new(completion: str) -> str:
    pattern_1 = re.compile(r"```python\n(.*?)```", re.DOTALL)
    pattern_2 = re.compile(r"```\n(.*?)```", re.DOTALL)
    matches = pattern_1.findall(completion) + pattern_2.findall(completion)
    if matches:
        extracted_answer = matches[0]
        extracted_answer = extract_function_body_new(extracted_answer)
    else:
        extracted_answer = completion
    return str(extracted_answer)


def extract_function_body_new(code: str) -> str:
    try:
        tree = ast.parse(code)
        for node in tree.body:
            if not isinstance(node, ast.FunctionDef):
                continue
            code_lines = code.splitlines()
            start = node.body[0].lineno - 1
            end = node.body[-1].end_lineno
            body_lines = code_lines[start:end]
            return "\n".join(body_lines)

    except Exception as e:
        print(f"Error extracting function body: {e}")
        return "error msg"
    

@scorer(metrics=[accuracy(), stderr()])
def verify() -> Scorer:
    async def score(state: TaskState, target: Target) -> Score:
        idx = state.sample_id
        model_completion = state.output.completion
        processed_completion = find_code_new(model_completion)

        val = CORRECT
        if 'error' in processed_completion:
            print(f'error in sample: {idx}')
            val = INCORRECT

        final_code = [
            state.metadata["prompt"],
            processed_completion,
            "\n",
            state.metadata["test"],
        ]

        return Score(
            value=val,
            metadata={
                'completion': model_completion,
                'processed': processed_completion,
                'final_code': ''.join(final_code),
                'idx': idx
            },
            answer=f'model completion: {model_completion}',
        )

    return score